In [30]:
!pip install -q streamlit
!pip3 install openai langchain
!pip install --upgrade weaviate-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.8/214.8 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requests-2.27.1:
      Successfully uninstalled requests-2.27.1


In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 3.106s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [61]:
%%writefile main.py

"""Python file to serve as the frontend"""
import streamlit as st

from langchain.chains import ConversationChain
from langchain.llms import OpenAI
import json
import weaviate
from bs4 import BeautifulSoup
from pprint import pprint
from langchain.prompts import PromptTemplate
from langchain import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain

WEAVIATE_CLUSTER = "ENTER HERE"


def load_llm(openai_key):
  """Logic for loading the chain you want to use should go here."""
  llm = OpenAI(temperature=0.1, 
            model_name='gpt-4',
            openai_api_key=openai_key)
  return llm

# From here down is all the StreamLit UI.
st.set_page_config(page_title="PGx Medication Regimen Review", page_icon=":robot:")
st.header("PGx Medication Regimen Review")

if "generated" not in st.session_state:
    st.session_state["generated"] = []

if "past" not in st.session_state:
    st.session_state["past"] = []


def get_patient_characteristics():
    input_text = st.text_area("Patient Characteristics: ", 
                              "", 
                              key="patient_characteristics")
    return input_text


def get_openai_key():
    input_text = st.text_input("OPENAI API KEY: ", 
                               "",
                               key="openai_key",
                               type="password")
    return input_text

def apply_guideline(llm, characteristics, guideline):
  prompt = PromptTemplate(
    input_variables=["guideline", "characteristics"],
    template="""You are an excellent clinician. You never make anything up.
    You have a patient with the following characteristics:
    {characteristics}

    Please apply the following algorithm to that patient:
    {guideline}
    """,
  )
  
  prompt_text = prompt.format(guideline=guideline,
                              characteristics=characteristics)

  print(f"\n{prompt_text}:")
  return llm(prompt_text)


patient_characteristics = get_patient_characteristics()
openai_key = get_openai_key()


# Every form must have a submit button.
submitted = st.button("Submit")
if submitted:
  if patient_characteristics and openai_key:
    client = weaviate.Client(
      url=WEAVIATE_CLUSTER,
      # auth_client_secret=weaviate.auth.AuthApiKey(api_key=WEAVIATE_KEY),
      additional_headers={
          "X-OpenAI-Api-Key": openai_key
      }
    )

    nearText = {"concepts": [patient_characteristics]}

    result = (
        client.query
        .get("Guideline", ["name", "relatedGenes", "guideline", "relatedChemicals", "algorithm"])
        .with_near_text(nearText)
        .with_limit(2)
        .do()
    )

    llm = load_llm(openai_key)

    for guideline in result['data']['Get']['Guideline']:
      st.info(guideline['algorithm'], icon="ℹ️")
      st.success(apply_guideline(llm, 
                                 patient_characteristics, 
                                 guideline['algorithm']), 
                 icon="✅")

    


  # st.session_state.past.append(user_input)
  # st.session_state.generated.append(output)

# if st.session_state["generated"]:

#     for i in range(len(st.session_state["generated"]) - 1, -1, -1):
#         message(st.session_state["generated"][i], key=str(i))
#         message(st.session_state["past"][i], is_user=True, key=str(i) + "_user")

Overwriting main.py


In [62]:
!streamlit run main.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 1.963s
your url is: https://long-doodles-exist.loca.lt
